In [ ]:
import pandas as pd
from myUtils import list_dump
path_dir = "C:/Users/ping/Desktop/my_yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
### edit file name to Barchart 52-week_new-highs ####
path_fn_52wkHi = "C:/Users/ping/Downloads/all-us-exchanges-52-week-new-highs-08-24-2022.csv"
#######################

In [ ]:
df = pd.read_csv(path_fn_52wkHi)  # read Barchart 52-week_new-highs
# drop last row (e.g. Downloaded from Barchart.com as of 08-24-2022 06:21pm CDT)
df.drop(df.tail(1).index,inplace=True)
df_shape_orig = df.shape
# create column of Last * Volume
df['Last*Vol'] = df.Last * df.Volume

In [ ]:
# apply filter Last*Vol > $10mil, Last > $10
df = df[df['Last*Vol'] > 10000000] 
df = df[df['Last'] > 10] 
df_shape_after = df.shape
row_deleted = df_shape_orig[0] - df_shape_after[0]
print(f'number of symbols removed by filter: {row_deleted}')
print(f'number of symbols remain:            {df_shape_after[0]}')

In [ ]:
# write symbols to text file
symbols_52wkHi = df['Symbol'].to_list()
list_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi.txt', verbose=True)

In [ ]:
# columns for df_52wkHi
myCols = ['symbol',
 'quoteType',
 'longName',
  'sector',
 'industry',
 'marketCap',
 'revenueGrowth',
 'ebitdaMargins',
 'earningsGrowth',
 'earningsQuarterlyGrowth',
  'previousClose',
  'volume',
 'fiftyTwoWeekHigh',
 '52WeekChange',
 'revenuePerShare',
 'forwardEps',
 'trailingEps',
 'sharesOutstanding']

In [ ]:
import yfinance as yf
# get yf.Ticker(symbol).info for keys in myCols for 52wkHi symbols 
df_list = []
for symbol in symbols_52wkHi:
    df_list.append(pd.DataFrame([yf.Ticker(symbol).info]))
df = pd.concat(df_list)
df = df[myCols].copy()

In [ ]:
from datetime import date
# put today's date in the date column
today = str(date.today())
df['date'] = today

In [ ]:
from myUtils import pickle_dump
# sort df_52wkHi with highest revenueGrowth on the 1st row
df_52wkHi = df.sort_values(by=['revenueGrowth', 'ebitdaMargins', 'earningsQuarterlyGrowth', 'earningsGrowth' ], ascending=False)
pickle_dump(df_52wkHi, path_data_dump, 'df_52wkHi')

In [ ]:
# apply filter revenueGrowth > .50 & ebitdaMargins > .25
df_filtered = df_52wkHi.query('revenueGrowth > .50 & ebitdaMargins > .25')
df_filtered


In [ ]:
symbols_52wkHi = df_filtered['symbol'].tolist()  # convert pd.Series to list
pickle_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi')

In [ ]:
# plot symbols
from myUtils import yf_candlestick
plot_days = '252d'
for symbol in symbols_52wkHi:
  df = yf.Ticker(symbol).history(period=plot_days)
  df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)  # drop Div. and Stock Splits columns
  df.columns = ['open', 'high', 'low', 'close', 'volume']  # rename columns Open ... Volume to open ... volume
  cache_plot = yf_candlestick(symbol, df, plot_chart=True)

In [ ]:
# print symbols data
from myUtils import yf_print_symbol_data
cache_print = yf_print_symbol_data(symbols_52wkHi)